In [1]:
import av

from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf

import keras
from keras import layers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout, Layer
from keras.optimizers import Adam
from keras import backend as K

import os
import random
import math
import numpy as np
import cv2

In [2]:
DS_CDFV1 = 'celeb_df_v1/'
DS_CDFV2 = 'celeb_df_v2/'

DS_ORGINAL = 'dataset_original/'
DS_SPLIT = 'dataset_split/'
DS_IFRAMES = 'dataset_iframes/'
DS_FACE = 'dataset_face/'
DS_FACE_IMG = 'dataset_face_img/'
DS_SRM_SNIPPETS = 'dataset_srm_snippets_5/'
DS_SEGMENTS = 'dataset_segments/'
DS_RAW = 'dataset_raw/'
DS_RESIDUALS = 'dataset_residuals/'

SEG_1 = 'seg_1/'
SEG_2 = 'seg_2/'
SEG_3 = 'seg_3/'
SEG_4 = 'seg_4/'
SEG_5 = 'seg_5/'

DS_TRAIN = 'train_dataset/'
DS_TEST = 'test_dataset/'
DS_VAL = 'val_dataset/'

CLASS_FAKE = 'fake/'
CLASS_REAL = 'real/'


TOP_LEVEL_1 = [DS_SPLIT, DS_IFRAMES, DS_FACE, DS_FACE_IMG, DS_SRM_SNIPPETS]
TOP_LEVEL_2 = [DS_SEGMENTS, DS_RAW, DS_RESIDUALS]
SEGMENTS = [SEG_1, SEG_2, SEG_3, SEG_4, SEG_5]
SPLIT = [DS_TRAIN, DS_TEST, DS_VAL]
CLASS = [CLASS_REAL, CLASS_FAKE]

DATASET = [DS_CDFV1, DS_CDFV2]

# Snippet Extraction

## Functions

In [100]:
# Returns the index of frames that begin a new segment (except the first segment)
def get_segment_dividers(frame_count, num_segments):
    segments_per_frame = math.floor(frame_count / num_segments)

    return [(segments_per_frame * i) for i in range(1, num_segments) ]

In [101]:
# Returns the indices of the frames that will be randomly selected from each segment
# Multiple snippets indices per segment can be returned by setting the num_snippets arg 
def get_snippet_indices(segment_dividers, num_snippets):
    start_index = 0
    num_snippets = 1 if num_snippets <= 0 else num_snippets

    snippet_indices = []
    for end_index in segment_dividers:

        # Extracting multiple snippets per segment (if needed)
        for _ in range(num_snippets):
            snippet_indices.append(random.randint(start_index, end_index - 1))

        start_index = end_index
        
    return snippet_indices

In [102]:
# Returns an array of randomly selected snippets(PIL.Image) from each segment of the input video
def extract_snippets(fp, num_segments, num_snippets):
    vid_container = av.open(fp)
    vid_stream = vid_container.streams.video[0]
    frame_count = vid_stream.frames

    snippets = []

    # If number of frames in video is less than the number of frames that need to sampled
    # then take all frames in the video
    if frame_count < num_segments * num_snippets:
        for frame in vid_container.decode():
            snippets.append(frame.to_image())

    else:
        segment_dividers = get_segment_dividers(frame_count, num_segments)
        segment_dividers = segment_dividers + [frame_count]

        snippet_indices = get_snippet_indices(segment_dividers, num_snippets)

        frame_index = 0
        for frame in vid_container.decode():
            if frame_index > max(snippet_indices):
                break

            if frame_index in snippet_indices:
                snippets.append(frame.to_image())

            frame_index += 1

    return snippets

## Testing Logic

In [54]:
tmp_count = 30
tmp_seg = get_segment_dividers(tmp_count, 3)
tmp_snip = get_snippet_indices(tmp_seg + [tmp_count], 2)

print(f'Segment Dividers: {tmp_seg}')
print(f'Snippets{tmp_snip}')

Segment Dividers: [10, 20]
Snippets[4, 9, 15, 13, 20, 26]


In [99]:
# test_file = os.listdir(DS_CDFV1 + DS_SPLIT + DS_TRAIN + CLASS_REAL)[0]
test_file = DS_CDFV2 + DS_SPLIT + DS_TRAIN + CLASS_REAL + 'id27_0005.mp4'
# test_input = av.open(os.path.realpath(DS_CDFV1 + DS_SPLIT + DS_TRAIN + CLASS_REAL + test_file))
test_input = av.open(test_file)

print(test_input.streams.video[0].frames)

# for frame in test_input.decode():
#     print(frame.key_frame)

1


In [95]:
# test_file = os.listdir(DS_CDFV1 + DS_FACE + DS_TRAIN + CLASS_REAL)[0]
test_file = DS_CDFV2 + DS_FACE + DS_TRAIN + CLASS_REAL + 'id27_0005.mp4'
# test_file = DS_CDFV1 + DS_FACE + DS_TRAIN + CLASS_REAL + test_file
tmp_snippets = extract_snippets(test_file, 5, 1)

for s in tmp_snippets:
    s.show()

## Implementation

### Celeb-DF v1 & v2

In [103]:
def save_snippets_CDF(dataset, num_segments, num_snippets):
    if dataset != DS_CDFV1 and dataset != DS_CDFV2:
        print(dataset)
        return
    
    random.seed(1)
    
    src_base_path = dataset + DS_FACE
    dst_base_path = dataset + DS_SRM_SNIPPETS

    for split in SPLIT:
        print(f'---Split started: {split}---')
        for class_dir in CLASS:
            print(f'Class started: {class_dir}')

            for video in os.listdir(src_base_path + split + class_dir):
                fp = src_base_path + split + class_dir + video
                snippets = extract_snippets(fp, num_segments, num_snippets)

                for i, snippet in enumerate(snippets, start=1):
                    seg_index = math.ceil(float(i) / num_snippets)
                    snip_index = (i - 1) % num_snippets
              
                    dst = f'{dst_base_path + split + class_dir + os.path.splitext(video)[0]}_s{seg_index}_f{snip_index}.jpeg'
                    snippet.save(dst)         

In [85]:
# CELEB DF V1
save_snippets_CDF(DS_CDFV1, num_segments=5, num_snippets=1)

---Split started: train_dataset/---
Class started: real/
Class started: fake/
---Split started: test_dataset/---
Class started: real/
Class started: fake/
---Split started: val_dataset/---
Class started: real/
Class started: fake/


In [104]:
# CELEB DF V2
save_snippets_CDF(DS_CDFV2, num_segments=5, num_snippets=1)

---Split started: train_dataset/---
Class started: real/
Class started: fake/
---Split started: test_dataset/---
Class started: real/
Class started: fake/
---Split started: val_dataset/---
Class started: real/
Class started: fake/


# Tensor Dataset Creation

In [3]:
def create_tensor_dataset(dataset, split):
    ds = keras.utils.image_dataset_from_directory(
        directory = dataset + DS_SRM_SNIPPETS + split,
        labels = 'inferred',
        label_mode = 'binary',
        batch_size = 32,
        color_mode = 'rgb',
        shuffle = True,
        seed = 1
    )

    return ds

## Celeb DF v1

In [4]:
train_dataset_cdfv1 = create_tensor_dataset(DS_CDFV1, DS_TRAIN)
test_dataset_cdfv1 = create_tensor_dataset(DS_CDFV1, DS_TEST)
val_dataset_cdfv1 = create_tensor_dataset(DS_CDFV1, DS_VAL)

Found 4415 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Found 1100 files belonging to 2 classes.


## Celeb DF v2

In [39]:
train_dataset_cdfv2 = create_tensor_dataset(DS_CDFV2, DS_TRAIN)
test_dataset_cdfv2 = create_tensor_dataset(DS_CDFV2, DS_TEST)
val_dataset_cdfv2 = create_tensor_dataset(DS_CDFV2, DS_VAL)

Found 24046 files belonging to 2 classes.
Found 2590 files belonging to 2 classes.
Found 6005 files belonging to 2 classes.


# Model Creation

## Functions

In [4]:
def init_srm_filters(shape, dtype=None):
    kernel1 = np.array([[0, 0,  0, 0, 0],
                        [0, 0,  0, 0, 0],
                        [0, 1, -2, 1, 0],
                        [0, 0,  0, 0, 0],
                        [0, 0,  0, 0, 0]], dtype=float)
    
    kernel2 = np.array([[0,  0,  0,  0, 0],
                        [0, -1,  2, -1, 0],
                        [0,  2, -4,  2, 0],
                        [0, -1,  2, -1, 0],
                        [0,  0,  0,  0, 0]], dtype=float)

    kernel3 = np.array([[-1,  2,  -2,  2, -1],
                        [ 2, -6,   8, -6,  2],
                        [-2,  8, -12,  8, -2],
                        [ 2, -6,   8, -6,  2],
                        [-1,  2,  -2,  2, -1]], dtype=float)
    
    
    kernel1 /= np.ones((5, 5), dtype=float) * 2
    kernel2 /= np.ones((5, 5), dtype=float) * 4
    kernel3 /= np.ones((5, 5), dtype=float) * 12

    k1_3D = np.dstack([kernel1, kernel1, kernel1])
    k2_3D = np.dstack([kernel2, kernel2, kernel2])
    k3_3D = np.dstack([kernel3, kernel3, kernel3])

    final_kernel = np.stack([k1_3D, k2_3D, k3_3D], axis=-1)
    return tf.convert_to_tensor(final_kernel, dtype=dtype)

In [32]:
class SRMLayer(Layer):
    def __init__(self, filters, kernel_size, fixed_filters, **kwargs):
        super(SRMLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.fixed_filters = fixed_filters

        self.q_small = self.add_weight(name='q_small',
                                       shape=(),
                                       initializer=keras.initializers.Constant(value=2.0),
                                       trainable=True)
        
        self.q_med = self.add_weight(name='q_med',
                                     shape=(),
                                     initializer=keras.initializers.Constant(value=4.0),
                                     trainable=True)
        
        self.q_large = self.add_weight(name='q_large',
                                       shape=(),
                                       initializer=keras.initializers.Constant(value=12.0),
                                       trainable=True)
        
        def build(self, input_shape):
            super(SRMLayer, self).build(input_shape)
            fixed_filters = tf.constant(self.fixed_filters, dtype=tf.float32)
            print(fixed_filters)

        def call(self, inputs):
            fixed_filters = tf.constant(self.fixed_filters, dtype=tf.float32)

            # print(fixed_filters[:, :, 0:1, :])

            output1 = tf.nn.conv2d(inputs, fixed_filters[:, :, 0:1, :], strides=[1, 1, 1, 1], padding='SAME')
            output2 = tf.nn.conv2d(inputs, fixed_filters[:, :, 1:2, :], strides=[1, 1, 1, 1], padding='SAME')
            output3 = tf.nn.conv2d(inputs, fixed_filters[:, :, 2:3, :], strides=[1, 1, 1, 1], padding='SAME')

            output1 /= self.q_small
            output2 /= self.q_med
            output3 /= self.q_large

            output = tf.concat([output1, output2, output3], axis=3)
            return output
        
        def compute_output_shape(self, input_shape):
            batch_size = input_shape[0]
            height = input_shape[1]
            width = input_shape[2]

        def get_config(self):
            config = super(SRMLayer, self).get_config()
            config.update({'filters': self.filters,
                           'kernel_size': self.kernel_size,
                           'fixed_filters': self.fixed_filters})
            
            return config


## Testing Logic

**For Convolution**
- Input Shape:  [batch_size, height, width, channels]
- Filter Shape: [height, width, in_channels, filter_count]

In [4]:
tmp_filter_small = tf.constant([[0, 0,  0, 0, 0],
                                [0, 0,  0, 0, 0],
                                [0, 1, -2, 1, 0],
                                [0, 0,  0, 0, 0],
                                [0, 0,  0, 0, 0]], dtype=tf.float32)

tmp_filter_med = tf.constant([[0,  0,  0,  0, 0],
                              [0, -1,  2, -1, 0],
                              [0,  2, -4,  2, 0],
                              [0, -1,  2, -1, 0],
                              [0,  0,  0,  0, 0]], dtype=tf.float32)

tmp_filter_small = tf.expand_dims(tf.expand_dims(tmp_filter_small, axis=-1), axis=-1)
tmp_filter_med   = tf.expand_dims(tf.expand_dims(tmp_filter_med,   axis=-1), axis=-1)
# tf.print(tmp_filter_small.shape)

tmp_filter_small = tf.tile(tmp_filter_small, [1, 1, 3, 1])
tmp_filter_med   = tf.tile(tmp_filter_med,   [1, 1, 3, 1])

tmp_filters = tf.concat([tmp_filter_small, tmp_filter_med], axis=3)

tf.print(tmp_filters.shape)

tmp_conv = Conv2D(filters=2, 
                  kernel_size=5, 
                  kernel_initializer=tf.keras.initializers.Constant(tmp_filters))

tmp_x = tf.random.normal([32, 256, 256, 3])
tmp_y = tmp_conv(tmp_x)

print(tmp_y.shape)

TensorShape([5, 5, 3, 2])
(32, 252, 252, 2)


### Old

In [15]:
class TSRM(Layer):
    def __init__(self, **kwargs):
        super(TSRM, self).__init__(**kwargs)

        self.filter_small = tf.constant([[0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 1, -2, 1, 0],
                                         [0, 0,  0, 0, 0],
                                         [0, 0,  0, 0, 0]], dtype=tf.float32)
        
        self.q_small = self.add_weight(shape=(),
                                       initializer='ones',
                                       dtype=tf.float32,
                                       trainable=True)
        
    def call(self, inputs):
        filter_small = self.filter_small / self.q_small

        # Input shape: [batch_size, height, width, channels]
        # Filter shape: [height, width, in_channels, out_channels]

        conv_small = tf.nn.conv2d(inputs, 
                                  tf.expand_dims(tf.expand_dims(filter_small, axis=-1), axis=-1), 
                                  strides=[1, 1, 1, 1], 
                                  padding='SAME')
        
        # conv_meddd = tf.nn.conv2d(inputs, filter_small, strides=[1, 1, 1, 1], padding='SAME')

        # outputs = tf.concat([conv_small, conv_meddd], axis=-1)
        return conv_small

In [16]:
tmp_model = keras.Sequential([
    keras.layers.Input(shape=(256, 256, 3)),
    TSRM(),
    Flatten(),
    Dense(1, activation='sigmoid')
])

tmp_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

ValueError: Exception encountered when calling layer "tsrm_5" (type TSRM).

in user code:

    File "C:\Users\Arun TK\AppData\Local\Temp\ipykernel_4440\1975148752.py", line 22, in call  *
        conv_small = tf.nn.conv2d(inputs,

    ValueError: Depth of output (1) is not a multiple of the number of groups (3) for '{{node tsrm_5/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, tsrm_5/ExpandDims_1)' with input shapes: [?,256,256,3], [5,5,1,1].


Call arguments received by layer "tsrm_5" (type TSRM):
  • inputs=tf.Tensor(shape=(None, 256, 256, 3), dtype=float32)

In [ ]:
tmp_history = tmp_model.fit(train_dataset_cdfv1, epochs=1)

In [34]:
tmp_kernel1 = np.array([[0, 0,  0, 0, 0],
                    [0, 0,  0, 0, 0],
                    [0, 1, -2, 1, 0],
                    [0, 0,  0, 0, 0],
                    [0, 0,  0, 0, 0]], dtype=float)

tmp_kernel2 = np.array([[0,  0,  0,  0, 0],
                    [0, -1,  2, -1, 0],
                    [0,  2, -4,  2, 0],
                    [0, -1,  2, -1, 0],
                    [0,  0,  0,  0, 0]], dtype=float)

tmp_kernel3 = np.array([[-1,  2,  -2,  2, -1],
                    [ 2, -6,   8, -6,  2],
                    [-2,  8, -12,  8, -2],
                    [ 2, -6,   8, -6,  2],
                    [-1,  2,  -2,  2, -1]], dtype=float)

tmp_fixed_kernals = [tmp_kernel1, tmp_kernel2, tmp_kernel3]
tmp_custom_layer = SRMLayer(filters=3, kernel_size=(5, 5), fixed_filters=tmp_fixed_kernals)

## Implementation